<a href="https://colab.research.google.com/github/amarildo86/datascience/blob/master/Digio_Hackathon_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Financial Data Analytics Hackathon</center>

### Contexto do desafio

Quando os bancos digitais começaram a nascer no Brasil, realizar um bom atendimento, solucionar problemas online, ter de graça a anuidade no cartão de crédito, os saques e as transferências eram pilares de uma proposta de valor inovadora. Mais recentemente, o que antes era uma diferenciação se tornou um padrão para qualquer conta digital. Diferentes produtos atualmente são criados, como programas de fidelidade, investimentos e empréstimos instantâneos para trazer mais simplicidade e atratividade para os clientes.

Ao mesmo tempo, trocar de banco ficou tão fácil quanto começar uma nova série no canal de Streaming. Em apenas sete minutos você pode se cadastrar em um novo banco e em até 18 horas estar aprovado¹. Essa rapidez e agilidade incrementou significativamente a exposição dos clientes aos serviços financeiros como a conta, crédito e cartões.

Essa competição, cada vez mais acirrada neste novo cenário, faz com que bancos digitais tenham um grande desafio, principalmente relacionado à rentabilidade dos clientes no produto cartão de crédito. Pensando na otimização financeira deste produto, há algumas perguntas-chave:

• Qual o melhor modelo para calcular a rentabilidade destes clientes desde a sua origem?

• Entre tantas ferramentas e métodos de análise de dados, como gerar o melhor modelo financeiro de Safra de Cartões respeitando os conceitos e regras de uma demonstração de resultados (DRE), Payback e otimizando ao máximo esforço e tempo?

### Problema de negócio:
*   Previsão da linhas da DRE que compõe a Margem Operacional
*   Item da lista



---



In [7]:
#libs principais para análise de dados
import pandas as pd
import numpy as np

#libs para tratamento de datas/séries temporais
from datetime import datetime
from datetime import date

#lib para estatística
import statsmodels.api as sm

#libs para deep learning
import tensorflow as tf
import keras

#lib para limpar avisos desnecessários
import warnings
warnings.filterwarnings(action='ignore')

In [11]:
#lib para carregar dados do google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#carregamento dos três dataset fornecidos
clientes = pd.read_csv("drive/MyDrive/digio_hackathon/clientes.csv", sep=';')
transacional = pd.read_csv("drive/MyDrive/digio_hackathon/transacional.csv", sep=';')
ml_predictions = pd.read_csv("drive/MyDrive/digio_hackathon/ml_predictions.csv", sep=';')